# !!!IMPORTANT SETUP!!!

## from your GoogleDrive:
1.   create a folder ProjectAML in /content/drive/MyDrive
2.   upload in ProjectAML (the modified version of) python code @ https://github.com/shyam671/AnomalySegmentation_CourseProjectBaseCode
3. create an empty folder /ProjectAML/datasets/Validation_Dataset
4.  import manually all the test dataset in /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset from https://drive.google.com/file/d/1r2eFANvSlcUjxcerjC8l6dRa0slowMpx/view, folder must contain subfolder of the zip (FS_LostFound_full, fs_static, RoadAnomaly, RoadAnomaly21, RoadObstacles)
5. create an empty folder /ProjectAML/datasets/Train_Dataset

## NOTE: training use a high capacity RAM -> Colab Pro, but you can skip since exist a pre-trained model

# **Install required modules**

In [ ]:
#do once
!pip3 install 'torch'
!pip3 install 'torchvision'
#optimized version of Pillow
!pip3 install 'Pillow-SIMD'
!pip3 install 'numpy'
!pip3 install 'matplotlib'
#required to work, don't use --visualize
!pip3 install 'visdom'
#script to download train dataset
!pip3 install 'cityscapesscripts'
#restart after import

# **Mount drive**

In [ ]:
import torch
#use GPU if available
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #'cpu' # 'cuda' or 'cpu'
print(DEVICE)

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!unzip -q /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/Validation_Dataset.zip \
 -d /content/drive/MyDrive/ProjectAML/datasets/

# **Import and reorganize cityscapes dataset**
### necessary to run IoU evaluation on val set

In [ ]:
#do once
import os
#switch to cpu for download
DEVICE = torch.device("cpu")

!rm -r /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset
!mkdir /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset

#register to https://www.cityscapes-dataset.com and login here
#241MB
!csDownload gtFine_trainvaltest.zip -d /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset
#register to https://www.cityscapes-dataset.com and login here or use saved credentials
#11GB
!csDownload leftImg8bit_trainvaltest.zip -d /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset

#unzip dataset
!unzip -q /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/gtFine_trainvaltest.zip \
 -d /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/gtFine_trainvaltest
!unzip -q /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit_trainvaltest.zip \
 -d /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit_trainvaltest



In [ ]:
#make sure that unzip complete successfully before executing this cell

#remove zip
!rm /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/gtFine_trainvaltest.zip
!rm /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit_trainvaltest.zip

In [ ]:

#move only necessary folder in /Train_Dataset
!mv /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/gtFine_trainvaltest/gtFine \
  /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/gtFine
!mv /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit_trainvaltest/leftImg8bit \
  /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit



In [ ]:
#make sure that mv complete successfully before executing this cell

#remove original folder (with README and other stuff)
!rm -r /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/gtFine_trainvaltest
!rm -r /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/leftImg8bit_trainvaltest

#set required OS env variable to run correctly csCreateTrainIdInstanceImgs
os.environ['CITYSCAPES_DATASET'] = '/content/drive/MyDrive/ProjectAML/datasets/Train_Dataset/'
#create label
!csCreateTrainIdLabelImgs

# return to gpu
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #'cpu' # 'cuda' or 'cpu'

In [ ]:
#synchronize your google drive after any modification to ProjectAML folder
drive.flush_and_unmount() #synch
drive.mount('/content/drive') #remount

Mounted at /content/drive


# **START TRAINING**
## skip, trained model already exists

In [ ]:
import os
start = os.getcwd()
folder_path = "/content/drive/MyDrive/ProjectAML/train"
os.chdir(folder_path)

!rm -r /content/drive/MyDrive/ProjectAML/save
!mkdir /content/drive/MyDrive/ProjectAML/save

%run main.py --savedir erfnet_train_cityscape \
 --datadir /content/drive/MyDrive/ProjectAML/datasets/Train_Dataset \
 --decoder --pretrainedEncoder /content/drive/MyDrive/ProjectAML/trained_models/erfnet_encoder_pretrained.pth.tar

#python -m visdom.server -port 8097 # run on local pc-> see result on localhost

# Eval AuPRC FPR95 on datasets

In [ ]:
!pip3 install 'ood-metrics'


In [ ]:
import os
start = os.getcwd()
folder_path = "/content/drive/MyDrive/ProjectAML/eval"
os.chdir(folder_path)
#modify parser: parse all the path elsewhere
%run eval_iou.py --datadir '/content/drive/MyDrive/ProjectAML/datasets/Train_Dataset' --subset val


# Run MSP, max logit, max entropy


In [ ]:
import os
start = os.getcwd()
folder_path = "/content/drive/MyDrive/ProjectAML/eval"
os.chdir(folder_path)



print("RoadAnomaly21 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly21/images/*.png' \
 --anomalyScore 'MSP'
print("RoadObstacle21 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadObsticle21/images/*.webp' \
 --anomalyScore 'MSP'
print("FS_LostFound_full MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/FS_LostFound_full/images/*.png' \
 --anomalyScore 'MSP'
print("fs_static MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/fs_static/images/*.jpg' \
 --anomalyScore 'MSP'
print("RoadAnomaly MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly/images/*.jpg' \
 --anomalyScore 'MSP'

print("RoadAnomaly21 max logit")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly21/images/*.png'
print("RoadObstacle21 max logit")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadObsticle21/images/*.webp'
print("FS_LostFound_full max logit")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/FS_LostFound_full/images/*.png'
print("fs_static max logit")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/fs_static/images/*.jpg'
print("RoadAnomaly max logit")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly/images/*.jpg'

print("RoadAnomaly21 max entropy")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly21/images/*.png' \
--anomalyScore 'maxEntropy'
print("RoadObstacle21 max entropy")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadObsticle21/images/*.webp' \
--anomalyScore 'maxEntropy'
print("FS_LostFound_full max entropy")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/FS_LostFound_full/images/*.png' \
--anomalyScore 'maxEntropy'
print("fs_static max entropy")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/fs_static/images/*.jpg' \
--anomalyScore 'maxEntropy'
print("RoadAnomaly max entropy")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly/images/*.jpg' \
--anomalyScore 'maxEntropy'


# Run temperature scaled MSP

In [ ]:
import os
start = os.getcwd()
folder_path = "/content/drive/MyDrive/ProjectAML/eval"
os.chdir(folder_path)

print("RoadAnomaly21 temp=0.5 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly21/images/*.png' \
 --anomalyScore 'tempMSP' --temp 0.5
print("RoadObstacle21 temp=0.5 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadObsticle21/images/*.webp' \
 --anomalyScore 'tempMSP' --temp 0.5
print("FS_LostFound_full temp=0.5 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/FS_LostFound_full/images/*.png' \
 --anomalyScore 'tempMSP' --temp 0.5
print("fs_static temp=0.5 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/fs_static/images/*.jpg' \
 --anomalyScore 'tempMSP' --temp 0.5
print("RoadAnomaly temp=0.5 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly/images/*.jpg' \
 --anomalyScore 'tempMSP' --temp 0.5

print("RoadAnomaly21 temp=0.75 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly21/images/*.png' \
 --anomalyScore 'tempMSP' --temp 0.75
print("RoadObstacle21 temp=0.75 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadObsticle21/images/*.webp' \
 --anomalyScore 'tempMSP' --temp 0.75
print("FS_LostFound_full temp=0.75 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/FS_LostFound_full/images/*.png' \
 --anomalyScore 'tempMSP' --temp 0.75
print("fs_static temp=0.75 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/fs_static/images/*.jpg' \
 --anomalyScore 'tempMSP' --temp 0.75
print("RoadAnomaly temp=0.75 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly/images/*.jpg' \
 --anomalyScore 'tempMSP' --temp 0.75

print("RoadAnomaly21 temp=1.1 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly21/images/*.png' \
 --anomalyScore 'tempMSP' --temp 1.1
print("RoadObstacle21 temp=1.1 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadObsticle21/images/*.webp' \
 --anomalyScore 'tempMSP' --temp 1.1
print("FS_LostFound_full temp=1.1 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/FS_LostFound_full/images/*.png' \
 --anomalyScore 'tempMSP' --temp 1.1
print("fs_static temp=1.1 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/fs_static/images/*.jpg' \
 --anomalyScore 'tempMSP' --temp 1.1
print("RoadAnomaly temp=1.1 MSP")
%run evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly/images/*.jpg' \
 --anomalyScore 'tempMSP' --temp 1.1

# Grid search of best temperature for MSP

In [ ]:
import os
from IPython import get_ipython
start = os.getcwd()
folder_path = "/content/drive/MyDrive/ProjectAML/eval"
os.chdir(folder_path)

def run_command(command):
    result = get_ipython().system(command)
    if result != 0:
        print(f"Error executing command: {command}")


for i in range(4, -5, -1):
    t = format(f"{10**i}")

    # Define the commands in a list
    commands = [
        f'echo "Running RoadAnomaly21 temp={t} MSP"',
        f"python3 evalAnomaly.py --input /content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly21/images/*.png --anomalyScore tempMSP --temp {t}",
        f'echo "Running RoadObstacle21 temp={t} MSP"',
        f"python3 evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadObsticle21/images/*.webp' --anomalyScore 'tempMSP' --temp {t}",
        f'echo "Running FS_LostFound_full temp={t} MSP"',
        f"python3 evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/FS_LostFound_full/images/*.png' --anomalyScore 'tempMSP' --temp {t}",
        f'echo "Running fs_static temp={t} MSP"',
        f"python3 evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/fs_static/images/*.jpg' --anomalyScore 'tempMSP' --temp {t}",
        f'echo "Running RoadAnomaly temp={t} MSP"',
        f"python3 evalAnomaly.py --input '/content/drive/MyDrive/ProjectAML/datasets/Validation_Dataset/RoadAnomaly/images/*.jpg' --anomalyScore 'tempMSP' --temp {t}"
    ]

    # Execute the commands using subprocess.run
    for command in commands:
        run_command(command)

